In [ ]:
##qSNAP MLIP
from monty.serialization import loadfn
from monty.serialization import dumpfn

In [ ]:
#import the training and test data from Json data folder
dict_list = loadfn('/Json_data_10842/8672train.json')
test_dict_list = loadfn('/Json_data_10842/1085test.json')

train_structures = [d['structure'] for d in dict_list]
train_energies = [d['outputs']['energy'] for d in dict_list]
train_forces = [d['outputs']['forces'] for d in dict_list]

test_structures = [d['structure'] for d in test_dict_list]
test_energies = [d['outputs']['energy'] for d in test_dict_list]
test_forces = [d['outputs']['forces'] for d in test_dict_list]

In [ ]:
import numpy as np
from maml.utils import pool_from, convert_docs

train_pool = pool_from(train_structures, train_energies, train_forces)
_, df = convert_docs(train_pool)

weights = np.ones(len(df['dtype']), )

# set the weights for energy equal to 100
weights[df['dtype'] == 'energy'] = 100

In [ ]:
from maml.base import SKLModel
from maml.describers import BispectrumCoefficients
from sklearn.linear_model import LinearRegression
from maml.apps.pes import SNAPotential

element_profile = {'Li': {'r': 5.0, 'w': 1}, 'Mn': {'r': 5.0, 'w': 1}, 'Ni': {'r': 5.0, 'w': 1}, 
                   'O': {'r': 5.0, 'w': 1}, 'Sc': {'r': 5.0, 'w': 1}, 'Ti': {'r': 5.0, 'w': 1},
                  'V': {'r': 5.0, 'w': 1}, 'Cr': {'r': 5.0, 'w': 1}, 'Fe': {'r': 5.0, 'w': 1},
                  'Co': {'r': 5.0, 'w': 1}, 'Cu': {'r': 5.0, 'w': 1}}
describer = BispectrumCoefficients(rcutfac=0.5, twojmax=12, element_profile=element_profile, 
                                   quadratic=True, pot_fit=True)
model = SKLModel(describer=describer, model=LinearRegression())
qsnap = SNAPotential(model=model)

import datetime
ct1 = datetime.datetime.now()

qsnap.train(train_structures, train_energies, train_forces, sample_weight=weights)

ct2 = datetime.datetime.now()

In [ ]:
#to get the erros and data in .csv file for training data
from sklearn.metrics import mean_absolute_error
import numpy as np

E_p = np.array(df_predict[df_predict['dtype'] == 'energy']['y_orig'])/df_predict[df_predict['dtype'] == 'energy']['n']
E_o = np.array(df_orig[df_orig['dtype'] == 'energy']['y_orig'])/df_orig[df_orig['dtype'] == 'energy']['n']
print("MAE of training energy prediction is {} meV/atom".format(mean_absolute_error(E_o,E_p)*1000))

F_p = np.array(df_predict[df_predict['dtype'] == 'force']['y_orig'])/df_predict[df_predict['dtype'] == 'force']['n']
F_o = np.array(df_orig[df_orig['dtype'] == 'force']['y_orig'])/df_orig[df_orig['dtype'] == 'force']['n']
print("MAE of training force prediction is {} eV/Å".format(mean_absolute_error(F_o,F_p)))

from sklearn.metrics import mean_squared_error
print("RMSE of training energy prediction is {} eV/atom".format(mean_squared_error(E_o,E_p,squared=False)))
print("RMSE of training force prediction is {} eV/atom".format(mean_squared_error(F_o,F_p,squared=False)))

df_predict.to_csv('qsnap_train_predict.csv')
df_orig.to_csv('qsnap_train_orig.csv')

In [ ]:
df_orig, df_predict = qsnap.evaluate(test_structures=test_structures, 
                                    test_energies=test_energies,
                                    test_forces=test_forces, test_stresses=None)

In [ ]:
#to get the erros and to save data in .csv file for training data
from sklearn.metrics import mean_absolute_error
import numpy as np

E_p = np.array(df_predict[df_predict['dtype'] == 'energy']['y_orig'])/df_predict[df_predict['dtype'] == 'energy']['n']
E_o = np.array(df_orig[df_orig['dtype'] == 'energy']['y_orig'])/df_orig[df_orig['dtype'] == 'energy']['n']
print("MAE of test energy prediction is {} meV/atom".format(mean_absolute_error(E_o,E_p)*1000))

F_p = np.array(df_predict[df_predict['dtype'] == 'force']['y_orig'])/df_predict[df_predict['dtype'] == 'force']['n']
F_o = np.array(df_orig[df_orig['dtype'] == 'force']['y_orig'])/df_orig[df_orig['dtype'] == 'force']['n']
print("MAE of test force prediction is {} eV/Å".format(mean_absolute_error(F_o,F_p)))

from sklearn.metrics import mean_squared_error
print("RMSE of test energy prediction is {} eV/atom".format(mean_squared_error(E_o,E_p,squared=False)))
print("RMSE of test energy prediction is {} eV/atom".format(mean_squared_error(F_o,F_p,squared=False)))

df_predict.to_csv('qsnap_test_predict.csv')
df_orig.to_csv('qsnap_test_orig.csv')